In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from langchain.llms import OpenAI
from langdetect import detect_langs
from langchain.agents import load_tools
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

In [ ]:

biden = pd.read_csv('data/hashtag_joebiden.csv', lineterminator='\n')
trump = pd.read_csv('data/hashtag_donaldtrump.csv', lineterminator='\n')

llm = OpenAI(temperature=0.2,
             openai_api_key=INSERT_API_KEY
             )

# Data Cleaning

In [4]:
def clean_and_filter_tweets(df, min_tweet_count=40, min_word_count=35):

    url_mention_hashtag_pattern = r'http\S+|@\S+|#\S+'

    df['cleaned_tweets'] = df['tweet'].str.replace(url_mention_hashtag_pattern,'', regex=True).str.strip()
    df = df[df['cleaned_tweets'].str.split().str.len() >= min_word_count]

    df = df.drop_duplicates(subset=['user_id', 'cleaned_tweets'])

    user_count = df['user_id'].value_counts()
    valid_users = user_count[user_count >= min_tweet_count].index
    df = df[df['user_id'].isin(valid_users)]

    return df[['user_id', 'cleaned_tweets']]



biden_cleaned = clean_and_filter_tweets(biden)
trump_cleaned = clean_and_filter_tweets(trump)

### Sampling and Naming

In [5]:
def sampling_and_naming(df, label, names):
    
    random_5 = df['user_id'].drop_duplicates().sample(n=5)
    sampled_df = df[df['user_id'].isin(random_5)].groupby('user_id').sample(n=25)
    name_map = dict(zip(random_5, [f"{label}_{name}" for name in names]))
    sampled_df['user_id'] = sampled_df['user_id'].map(name_map)
    return sampled_df

biden_random5 = sampling_and_naming(biden_cleaned, 'Biden', "ABCDE")
trump_random5 = sampling_and_naming(trump_cleaned, 'Trump', "FGHIJ")
df = pd.concat([biden_random5, trump_random5], ignore_index=True)



# Prompt Template

In [4]:
def mock_persona(tweet_examples, task):
    def get_authors_tone_description(tweet_examples):

        template = f"""
            You are an AI Bot skilled in extracting political stances from writing examples.
            Please analyze the examples provided and determine the political stance conveyed in the writing.

            % START OF EXAMPLES
            {tweet_examples}
            % END OF EXAMPLES

            Describe the political stance of the example above.
        """
        final_prompt = template.format(tweet_examples=tweet_examples)
        return llm.predict(final_prompt)
    
    template = \
        """
        % INSTRUCTIONS
        - You are an AI Bot that is very good at mimicking an author writing style.
        - Your goal is to write content with the political stance tone that is described below.
        - You're a smart AI. You can determine from the examples below whether the user is a supporter of Trump, Biden, or neither.
        - Do not go outside the tone instructions below

        % Description of the authors political stance tone:
        {authors_tone_description}

        % START OF EXAMPLES
        {tweet_examples}
        % END OF EXAMPLES

        % YOUR TASK
        1. Answering the question "{task}" using author writing style and mock author political persona
    """
    
    authors_tone_description = get_authors_tone_description(tweet_examples)
    prompt_template = \
        template.format(authors_tone_description=authors_tone_description,tweet_examples=tweet_examples, task=task)
    return prompt_template

### Agent

In [5]:
def main(user_letter, task, df=df):
    user_tweets = df[df['user_id'] == user_letter]['cleaned_tweets'].tolist()
    prompt_template = mock_persona(user_tweets, task)

    
    tools = load_tools(['ddg-search', 'llm-math'], llm=llm)
    agent = initialize_agent(tools, llm,
                             agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                             handle_parsing_errors=True,
                             verbose=False)

    return agent.run(prompt_template)

### Testing

In [6]:
main('Biden_B',
     """
     Base on your political stance, how do you view the events of January 6, 2021, at the U.S. Capitol, 
     and what were the factors that led to these events?
     """
     )

"The events of January 6, 2021, at the U.S. Capitol were a result of Donald Trump's refusal to accept the results of the 2020 presidential election, which Joe Biden had won. This refusal to accept the election results led to a mob of Trump's supporters attacking the U.S. Capitol as Congress met to certify the results. This attack was planned and executed by the mob, and was enabled by Trump's rhetoric and refusal to accept the election results."

### Political Questions & Answer

In [10]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you support Trump or Biden? Or you are neutral? ')
    print(f'{user_name}: {result}\n')

Biden_A: I support Donald Trump.

Biden_B: I support Joe Biden in the 2020 US Presidential Election.

Biden_C: I support Joe Biden.

Biden_D: I am neutral and do not support either Trump or Biden.

Biden_E: The author's political stance is strongly against the Democratic Party and its candidates, Joe Biden and Hunter Biden. They criticize the Democrats for attempting to delegitimize election results, cashing in on foreign sources, and using a younger, more agile Biden in ads. They also criticize the Biden campaign for failing to address the corruption allegations and for attempting to hide from the public. Additionally, the examples criticize the Democratic Party for its support of Black Lives Matter and Antifa, and for its failure to disclose millions of dollars from China. Therefore, the author supports Trump.

Trump_F: I do not support Donald Trump. I believe that he is a liar, a narcissist, and a dangerous person who is trying to divide the country. Furthermore, I believe that his 

In [12]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree with the United States improving LGBTQIA+ rights, with 1 indicating disagree, 2 indicating neutral, and 3 indicating agree? Tell us why did you feel that way. Please explain your choice in a sentence starts with your score and then your reasoning.')
    print(f'{user_name}: {result}\n')

Biden_A: Agent stopped due to iteration limit or time limit.

Biden_B: 3. I agree with the United States improving LGBTQIA+ rights because there is evidence of discrimination against LGBTQIA+ people in the United States, and the government is taking steps to address this discrimination.

Biden_C: 3 - Joe Biden is a strong supporter of LGBTQIA+ rights, as evidenced by his statements and initiatives to protect LGBTQIA+ communities from attacks on their rights and safety.

Biden_D: The examples provided do not convey a political stance, so I cannot answer the question with a score and reasoning.

Biden_E: 3. I agree with the United States improving LGBTQIA+ rights because I believe that everyone should be treated equally and with respect, regardless of their gender identity or sexual orientation.

Trump_F: 3. I agree with the United States improving LGBTQIA+ rights because Donald Trump has a history of disregarding the rights of LGBTQIA+ individuals and has been accused of attempting to r

In [13]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree with the United States increasing gun control, with 1 indicating disagree, 2 indicating neutral, and 3 indicating agree? Tell us why did you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')

Biden_A: I agree with increasing gun control, with a score of 3 indicating agreement. I believe that gun control is necessary in order to reduce the number of mass shootings in America and to protect the safety of citizens. Gun control measures such as background checks, waiting periods, and restrictions on certain types of firearms can help to ensure that guns are not falling into the wrong hands.

Biden_B: 3 - I agree with increasing gun control because President Joe Biden has consistently spoken out against former President Donald Trump's rhetoric and policies, and has made it clear that he believes in protecting the safety of the American people.

Biden_C: I agree with increasing gun control, with a score of 3 indicating agree. Joe Biden has been a strong advocate for gun control, and I believe that increasing gun control is necessary to reduce gun violence in the United States. Biden has also been vocal about the need for universal background checks, closing the gun show loophole,

In [14]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree with abortion being legalized in all 50 states, with 1 indicating disagree, 2 indicating neutral, and 3 indicating agree? Tell us why did you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: I agree with abortion being legalized in all 50 states, with 3 indicating agree. I believe that abortion should be a decision made between a woman and her doctor, and that the government should not be involved in this decision. I also believe that access to safe and legal abortions should be available to all women, regardless of their financial situation.

Biden_B: I agree with abortion being legalized in all 50 states, with a score of 3 indicating agreement. This is because I believe that women should have the right to make decisions about their own bodies, and that access to safe and legal abortions is a fundamental part of that right. Additionally, I believe that access to abortion is a necessary part of providing healthcare for women, and that it should be available to all women regardless of their economic or social status.

Biden_C: I agree with abortion being legalized in all 50 states, with a score of 3. I believe that the right to choose is a fundamental right that sh

In [17]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree with the universities in the United States practicing affirmative action, with 1 indicating disagree, 2 indicating neutral, and 3 indicating agree? Tell us why did you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: I agree with the universities in the United States practicing affirmative action, with a score of 3 indicating agreement. I believe that affirmative action is a necessary tool to ensure that everyone has an equal opportunity to succeed, regardless of their race, gender, or background. It is important to recognize and address the systemic inequalities that have been in place for centuries, and affirmative action is a step in the right direction.

Biden_B: I agree with the universities in the United States practicing affirmative action, scoring 3 on the scale. This is because the author's political stance is generally in favor of the Democratic candidate, Joe Biden, and the importance of the US election in international politics, and the criticism of the current President Donald Trump's handling of the virus and other issues. Affirmative action is a policy that is meant to help create more equitable opportunities for people who have been historically marginalized, and this is in

In [18]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree that climate change in the US is cause for concern, with 1 indicating disagree, 2 indicating neutral, and 3 indicating agree? Tell us why did you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: 3. I agree that climate change in the US is cause for concern. President Joe Biden has been vocal about the need to address climate change and has taken steps to do so, while former President Donald Trump has denied the existence of climate change and has rolled back many of the environmental regulations put in place by the Obama administration. It is clear that Biden is the better choice for those who are concerned about climate change.

Biden_B: I agree that climate change in the US is cause for concern, with a score of 3 indicating agree. The search results show that the impacts of a rapidly warming climate are being felt in every corner of the US and will worsen over the next 10 years as planet-warming pollution continues to rise. Additionally, the report finds that climate-fueled dangers like wildfires and heat waves are on the rise across the country, and that climate change causes tens of billions of dollars in economic damage in the United States every year. Therefore,

In [20]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree that the United States needs more paths for citizenship for undocumented immigrants, with 1 indicating disagreement, 2 indicating neutral, and 3 indicating agreement? Tell us why you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: Agent stopped due to iteration limit or time limit.

Biden_B: I agree with the author's stance on immigration and believe that the United States needs more paths for citizenship for undocumented immigrants. My score is 3, indicating agreement, because I believe that the United States should provide a safe and secure pathway to citizenship for undocumented immigrants, and that this would be beneficial for both the immigrants and the country as a whole.

Biden_C: 3. I agree that the United States needs more paths for citizenship for undocumented immigrants because Joe Biden supports more paths for citizenship for undocumented immigrants.

Biden_D: I agree with the United States needing more paths for citizenship for undocumented immigrants, scoring a 3 on the scale. President Biden's proposed U.S. Citizenship Act of 2021 is a step in the right direction, as it would establish a new immigration system to responsibly manage migration across the Western Hemisphere and expand lawful

In [22]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree that the death penalty should exist in the United States, with 1 indicating disagreement, 2 indicating neutral, and 3 indicating agreement? Tell us why you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: I agree with the Republican Party's stance on the death penalty and believe it should exist in the United States. My score is 3, indicating agreement, because I believe that the death penalty is a necessary part of the justice system and can be used as a deterrent to crime.

Biden_B: 3. I agree that the death penalty should exist in the United States, with a score of 3 indicating agreement. I believe that the death penalty is a necessary tool for justice and deterrence of crime, and that it should be used in cases of extreme violence and heinous crimes.

Biden_C: Agent stopped due to iteration limit or time limit.

Biden_D: I agree with the existence of the death penalty in the United States, with a score of 3 indicating agreement. I believe that the death penalty is a necessary tool for justice and deterrence, and that it should be used in cases of the most heinous crimes.

Biden_E: 3. I agree that the death penalty should exist in the United States. I believe that it is a ne

In [24]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree that the military should allow women in positions of combat in the United States, with 1 indicating disagreement, 2 indicating neutral, and 3 indicating agreement? Tell us why you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: Agent stopped due to iteration limit or time limit.

Biden_B: 3. I agree that the military should allow women in positions of combat in the United States. Women have been serving in the military in one form or another for more than 200 years, and there are many inspiring female military heroes. Allowing women in positions of combat is an important step in recognizing the contributions of women in the military.

Biden_C: 3 - I agree that the military should allow women in positions of combat in the United States. As President Biden has made it a top priority to eliminate obstacles to women's military service through unprecedented actions to promote their safety, inclusion, health and success, I believe that allowing women to serve in combat roles is an important step in the right direction.

Biden_D: The author is likely to be in favor of allowing women in positions of combat in the United States, as they appear to be a supporter of Marx's theories.

Biden_E: I agree that the m

In [25]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree that marijuana should be legalized in the United States, with 1 indicating disagreement, 2 indicating neutral, and 3 indicating agreement? Tell us why you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning.')
    print(f'{user_name}: {result}\n')




Biden_A: My score is 3, indicating agreement that marijuana should be legalized in the United States. I agree with this because the author's political stance is one of advocating for the people and their rights, which includes the right to make decisions about their own health and well-being. Furthermore, the author is critical of politicians for lying and engaging in unethical practices, which could include the prohibition of marijuana.

Biden_B: 3 - I agree that marijuana should be legalized in the United States. The majority of Americans, including Democrats, Independents, and Republicans, support marijuana legalization, and the Biden campaign should pivot to support it. Legalizing marijuana would bring numerous benefits, such as providing relief to those suffering from chronic pain and other medical conditions, creating jobs, and generating tax revenue.

Biden_C: I agree with the legalization of marijuana in the United States, scoring a 3 on the scale. President Biden has taken ste

In [27]:
user_names = ['Biden_A', 'Biden_B', 'Biden_C', 'Biden_D', 'Biden_E', 'Trump_F', 'Trump_G', 'Trump_H', 'Trump_I', 'Trump_J']

for user_name in user_names:
    result = main(user_name, 'Do you agree interest rates for student loans should be lowered in the United States, with 1 indicating disagreement, 2 indicating neutral, and 3 indicating agreement? Tell us why you feel that way. Please explain your choice starting with your score, and then in another sentence your reasoning')
    print(f'{user_name}: {result}\n')




Biden_A: 3. I agree that interest rates for student loans should be lowered in the United States. My score of 3 indicates agreement and my reasoning is that student loan debt is a major burden for many Americans and lowering interest rates would help alleviate this burden.

Biden_B: Agent stopped due to iteration limit or time limit.

Biden_C: I agree with lowering student loan interest rates in the United States, scoring a 3 on the scale. The Biden administration has already taken steps to support students and borrowers, including cancelling more than $116 billion in student loan debt and introducing the SAVE plan, which makes many borrowers' monthly payments as low as $0 and prevents them from defaulting. These actions demonstrate the Biden administration's commitment to making college more affordable and accessible for all Americans.

Biden_D: I would score 3 on the question of whether interest rates for student loans should be lowered in the United States. This is because public op

-----